Hemer Jan, Pavlovský Jakub
# Prove of concept
---
This code serves as a proof of feasibility of the project. We have succesfully managed to scrape data from the *www.fortunaliga.cz* webiste for 2022/23 football season. this is not the whole dataset as multiple other seasons will be also scraped and merged to the dataset, using a similar method.

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#If you don't have these packages installed, please install it, otherwise it won't work

In [101]:
list_number1 = ['150', '0', '50', '50', '150', '200', '250']
list_number2 = ['0', '50', '100', '150', '200', '250', '300']
col_names = ['Zápas', 'G', 'G1p', 'G2p', 'ÚT', 'GR', 'S', 'SnB', 'P', 'P%', 'ŽK', 'ČK', 'F']
merged_df = pd.DataFrame(columns = col_names)

for yea in range(2014, 2025):
    for ln1, ln2 in zip(list_number1, list_number2):
        response = requests.get(f"https://www.fortunaliga.cz/statistiky?unit=4&status=&parameter=1&season={yea}&club=0&game_limit=&nationality=&age=&position=&list_number={ln1}&order=2&order_dir=1&list_number={ln2}")
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_ = 'table')
        
            titles = soup.find_all('abbr')
            titles = [element.text for element in titles]

            content = table.find_all('tr')
            #print(type(content))
            content = [table.find_all('td') for element in content]
            content = [str(element.text) for result_set in content for element in result_set]
            #print(type(content))

            num_elements_per_row = 14
            num_rows = len(content) // num_elements_per_row
            rows = [content[i:i + num_elements_per_row] for i in range(0, len(content), num_elements_per_row)]
            df = pd.DataFrame(rows, columns = [f'Column_{i+1}' for i in range(num_elements_per_row)])
            df = df.set_index('Column_1')
            df.columns = titles
            df = df.reset_index()
            df = df.drop_duplicates(subset = ['Column_1'])

            merged_df = pd.concat([merged_df, df], axis = 0, ignore_index = True)

In [108]:
merged_df.tail()

,Zápas,G,G1p,G2p,ÚT,GR,S,SnB,P,P%,ŽK,ČK,F,Column_1
2729,"SLA-TEP 0:7, 13/14",7,2,5,140,7,34,11,822,"76,3%",7,1,28,236.
2730,"OLO-JIH 4:3, 13/14",7,2,5,167,1,36,17,842,"75,7%",6,0,28,237.
2731,"MBL-BRN 5:2, 13/14",7,4,3,160,3,29,16,821,"80,6%",1,0,26,238.
2732,"PLZ-FKJ 6:1, 13/14",7,4,3,157,5,38,13,731,"85,0%",7,1,32,239.
2733,"FKJ-ZNO 5:5, 13/14",10,4,6,163,0,38,17,856,"80,7%",7,0,32,240.
